# Import Required Libraries
Import pandas, numpy, and os for data manipulation and file operations.

In [ ]:
import pandas as pd
import numpy as np
import os

# Load the Airbnb Dataset
Read the Airbnb dataset from an Excel file. Handle errors if the file or directory does not exist.

In [ ]:
# Read the Airbnb dataset
try:
    Airbnb_df = pd.read_excel(os.path.join("Datasource", "airbnb.xlsx"))
    print("===== Retrieve original dataset successfully =====")
except Exception as e:
    print(f"Error reading dataset: {str(e)}")
    print("Please check if the 'Datasource' directory exists and you have read permissions.")

# Initial Data Exploration
Display the shape of the dataset and print all column names to understand the data structure.

In [ ]:
# Display the shape of the dataset
print(f"Dataset Overview: {Airbnb_df.shape[0]:,} x {Airbnb_df.shape[1]}")

# Print all column names
print("\nAll Column Names:")
for i, col in enumerate(Airbnb_df.columns, 1):
    print(f"{i:2d}. {col}")

# Sort and Describe Data
Sort the dataset by 'Host Id' and provide a statistical summary of the 'Price' column.

In [ ]:
# Sort by 'Host Id' and describe 'Price'
Airbnb_df = Airbnb_df.sort_values(by='Host Id', ascending=True)
print(f"Reviews per Price: {Airbnb_df['Price'].describe()}")
print(Airbnb_df.head(10))

# Clean Column Names
Remove leading and trailing spaces from all column names to ensure consistency.

In [ ]:
# Remove leading and trailing spaces from column names
for col in Airbnb_df.columns:
    clean_col = col.strip()
    if col != clean_col:
        Airbnb_df.rename(columns={col: clean_col}, inplace=True)

# Remove Duplicates
Drop duplicate rows based on 'Host Id' and 'Host Since' to keep unique host records.

In [ ]:
# Remove duplicate rows based on 'Host Id' and 'Host Since'
Airbnb_df = Airbnb_df.drop_duplicates(subset=['Host Id', 'Host Since'])

# Convert Data Types
Convert the 'Host Since' column to datetime format and keep only the date part.

In [ ]:
# Convert 'Host Since' to datetime and keep only the date part
Airbnb_df["Host Since"] = pd.to_datetime(Airbnb_df["Host Since"], format="%d/%m/%Y").dt.date

# Drop Unnecessary Columns
Remove the 'Review Scores Rating (bin)' column as it is not needed for further analysis.

In [ ]:
# Delete 'Review Scores Rating (bin)' column
Airbnb_df = Airbnb_df.drop(columns=['Review Scores Rating (bin)'])

# Ensure Numeric Columns
Convert 'Price', 'Number of Records', 'Number Of Reviews', and 'Review Scores Rating' columns to numeric types, coercing errors to NaN.

In [ ]:
# Ensure numeric columns are in the correct format
Airbnb_df['Price'] = pd.to_numeric(Airbnb_df['Price'], errors='coerce')
Airbnb_df['Number of Records'] = pd.to_numeric(Airbnb_df['Number of Records'], errors='coerce')
Airbnb_df['Number Of Reviews'] = pd.to_numeric(Airbnb_df['Number Of Reviews'], errors='coerce')
Airbnb_df['Review Scores Rating'] = pd.to_numeric(Airbnb_df['Review Scores Rating'], errors='coerce')

# Impute Missing Prices
Fill missing 'Price' values using the median price by 'Zipcode' and then by 'Neighbourhood'. Drop rows where 'Price' is still missing.

In [ ]:
# Fill empty/NaN values for 'Price' using median by 'Zipcode' and then by 'Neighbourhood'
Airbnb_df['Price'] = Airbnb_df['Price'].fillna(Airbnb_df.groupby('Zipcode')['Price'].transform('median'))
Airbnb_df['Price'] = Airbnb_df['Price'].fillna(Airbnb_df.groupby('Neighbourhood')['Price'].transform('median'))
Airbnb_df = Airbnb_df.dropna(subset=['Price'])

# Remove Outliers and Invalid Rows
Remove rows with 'Review Scores Rating' outside 1-100 and rows with no records. Drop rows with missing key fields.

In [ ]:
# Remove outliers and invalid rows
Airbnb_df = Airbnb_df[(Airbnb_df['Review Scores Rating'] >= 1) & (Airbnb_df['Review Scores Rating'] <= 100)]
Airbnb_df = Airbnb_df[(Airbnb_df['Number of Records'] >= 1)]
Airbnb_df = Airbnb_df.dropna(
    subset=[
        'Host Id', 'Host Since', 'Neighbourhood', 'Zipcode',
        'Property Type', 'Room Type', 'Beds', 'Price',
        'Number of Records', 'Number Of Reviews', 'Review Scores Rating'
    ]
)

# Categorize Columns
Convert 'Neighbourhood', 'Room Type', 'Beds', and 'Property Type' columns to categorical data types for better memory usage and analysis.

In [ ]:
# Convert columns to categorical data types
Airbnb_df['Neighbourhood'] = Airbnb_df['Neighbourhood'].astype('category')
Airbnb_df['Room Type'] = Airbnb_df['Room Type'].astype('category')
Airbnb_df['Beds'] = Airbnb_df['Beds'].astype('category')
Airbnb_df['Property Type'] = Airbnb_df['Property Type'].astype('category')

# Reorder Columns
Rearrange columns for better readability and easier analysis.

In [ ]:
# Reorder columns for better readability
Airbnb_df = Airbnb_df[['Host Id', 'Host Since', 'Neighbourhood', 'Zipcode', 'Property Type', 'Room Type', 'Beds', 'Price', 'Number of Records', 'Number Of Reviews', 'Review Scores Rating']]

# Save Cleaned Dataset
Save the cleaned dataset as a CSV file. Handle errors if the directory does not exist or is not writable.

In [ ]:
# Save the cleaned dataset with error handling
try:
    output_path = os.path.join("Datasource", "airbnb_clean.csv")
    Airbnb_df.to_csv(output_path, index=False)
    print(f"✅ Dataset saved successfully to: {output_path}")
    print(f"📊 Cleaned dataset: {Airbnb_df.shape[0]:,} rows × {Airbnb_df.shape[1]} columns")
except Exception as e:
    print(f"❌ Error saving dataset: {str(e)}")
    print("⚠️  Please check if the 'Datasource' directory exists and you have write permissions.")